In [1]:
from datasets import load_dataset, Dataset
from langchain.embeddings import HuggingFaceBgeEmbeddings
from qdrant_client import models, QdrantClient
from langchain.vectorstores import Qdrant
import time
from datetime import timedelta
import pandas as pd
from pandas import DataFrame

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TODO
- Turn off streaming
- Add counter
- Implement saving
- Put into script
- Make tables in MasterThesis
- Implement other scripts

## Overview of the dataset attributes
- split: To which part of the dataset it belongs (retrieval or llm training)
- query: The actual question or query
- prediction: The outcome it should provide
– context: The retrieved/provided context which is used as background
- src: The source of the question/the name of the dataset 
- id: The id of the sample, which does not have a true format
- context_src: The source of the context (src dataset/wiki/cc)
- context_id: The id of the context
- original_context: If the context is provided or retrieved
- task: The kind of task summ/qa/multiple choice
- domain: The domain of the question open-domain qa/rc/summ/cotr ...

In [43]:
col_name = "retriever"

#tokenizer = LlamaTokenizer.from_pretrained("../models/llama7b", device_map='cuda')
embedding = HuggingFaceBgeEmbeddings(model_name="../models/retriever/bge-base-en-v1.5", model_kwargs={"device": "cuda:0"})

# Create the retriever
client = QdrantClient(url="http://localhost:6333")
db = Qdrant(client, 
            collection_name=col_name,
            embeddings=embedding,
            )

def search(query: str, k: int = 3):
    success = False
    while not success:
        try:
            results = db.similarity_search_with_score(query, k=k)
            if results:
                success = True 
        except:
            print(f"Error with example {query}, retrying in 0.2s")
            time.sleep(0.2)
    return results

# This function creates an example with the query and the prediction and the top k results
def make_example(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, split = "llm", retrieval = True, task="", domain=""):
    contexts = []
    if retrieval:
        # Search for the query
        results = search(query, k=k)

        # Get the text of the results
        contexts = [{
            "text": result[0].page_content, 
            "src": result[0].metadata["src"] if "src" in result[0].metadata.keys() else "unknown", 
            "id": result[0].metadata["id"] if "id" in result[0].metadata.keys() else result[0].metadata["title"],
            "original_context": False } 
            for result in results]
    
    # Add the original context
    if context:
        contexts.append({
            "text": context, 
            "src": dataset_name, 
            "id": example_id, 
            "original_context": True
            })

    return { 
        "split": split, 
        "query": query, 
        "prediction": prediction, 
        "contexts": contexts, 
        "src": dataset_name, 
        "id": example_id,
        "task": task,
        "domain": domain, 
        }

# Depending if the examples should be searved consecutively or not (shuffling not possible with all contexts in one example)
# This function creates a list of examples with one for each context
def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True, task="", domain=""):
    examples = []
    ex = make_example(query, prediction, dataset_name, context, example_id, k, retrieval=retrieval, task=task, domain=domain)
    for c in ex["contexts"]:
        examples.append({ 
            "split": ex["split"], 
            "query": ex["query"], 
            "prediction": ex["prediction"], 
            "context": c["text"], 
            "src": ex["src"], 
            "id": ex["id"], 
            "context_src": c["src"], 
            "context_id": c["id"], 
            "original_context": c["original_context"],
            "task": task,
            "domain": domain,
            })
    return examples

rarit_dataset_buffer = []
rarit = DataFrame(columns=["split", "query", "prediction", "context", "src", "id", "context_src", "context_id", "original_context", "task", "domain"])

def save_example(i, start, last_time, example, dname, force=False):
    save_examples(i, start, last_time, [example], dname, force=force)

def save_examples(i, start, last_time, examples, dname, force=False):
    global rarit_dataset_buffer
    global rarit
    # Save the dataset to a file
    rarit_dataset_buffer.extend(examples)
    
    if i % 100 == 0 or force:
        current_time = time.time()
        print(f"Processed {i} {dname} examples, time: {str(timedelta(seconds=(last_time - start)))}, last 100 in {str(timedelta(seconds=(current_time - last_time)))}")
        last_time = current_time
        if len(rarit_dataset_buffer) > 0:
            if rarit.empty:
                rarit = DataFrame(rarit_dataset_buffer)
            else:
                df = DataFrame(rarit_dataset_buffer)
                rarit = pd.concat([rarit, df])
            rarit_dataset_buffer.clear()
        rarit.to_parquet("../data/dataset/rarit_dataset_llm.parquet")
        rarit.to_csv("../data/dataset/rarit_dataset_llm.csv")

### Maintainance and testing

In [44]:
col = client.get_collection(collection_name=col_name)
status = col.status
optimizer = col.optimizer_status
ivecs = col.indexed_vectors_count
vecs = col.points_count

print(f"Collection status: {status}") # If collection status green, then it is ready to be used
print(f"Optimizer status: {optimizer}")
print(f"Total vectors: {vecs}")
print(f"Indexed vectors: {ivecs}")

Collection status: green
Optimizer status: ok
Total vectors: 184329218
Indexed vectors: 184329218


In [13]:
# Set to 0 to disable indexing during upload
# Set to 20000 to enable indexing after upload - check the status with the cell above
client.update_collection(
    collection_name=col_name,
    optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000),
)

True

In [30]:
# The corpus is of questionble quality
# Try to filter out some of the bad documents by reranker
start = time.time()
search_res = db.similarity_search_with_score("When did Albert Einstein die?", k=10) # Alternatively "mml" for maximal marginal relevance
elapsed = (time.time() - start)
clock = str(timedelta(seconds=elapsed))

for i, (doc, score) in enumerate(search_res):
    print(f"{i+1}. {score:.2f} {doc.page_content[:100]}")
print(f"Time: {clock}")

1. 0.82 Albert Einstein (March 14, 1879-April 18, 1955) Albert Einstein was born March 14, 1879 in Ulm, Wurt, time: 0:00:00.050777
2. 0.81 Hans Albert Einstein died due to heart failure on July 26, 1973 (aged 69) in Woods Hole, Massachuset, time: 0:00:00.050777
3. 0.81 Just as the name Frankenstein elicits visions of monsters, the name Einstein is synonymous with geni, time: 0:00:00.050777
4. 0.80 Albert Einstein’s name is almost synonymous. leading him to fight racism in the U.S. Einstein died A, time: 0:00:00.050777
5. 0.79 unified fieldtheory, which however was not successful. Albert Einstein died onApril 18, 1955. He was, time: 0:00:00.050777
6. 0.79 Today we remember renowned physicist Albert Einstein, who died April 18, 1955 at the age of 76. Born, time: 0:00:00.050777
7. 0.78 note that my friend and colleague M. Besso steadfastly stood by me in my work on the problem here di, time: 0:00:00.050777
8. 0.78 Einstein died of heart trouble, in Manhattan, New York County, New York, on

In [13]:
# This should be the estimated upper bound for the dataset size
# With 10 samples in memory, 750 times more, about 20 datasets in total
# Times two to accommondate for outliers
DataFrame(rarit_dataset_buffer).memory_usage(deep=True).sum() * 750 * 20 * 2

# Resulting in about 1.3 GB of data


1291890000

# Building the Dataset for the LLM
In this Notebook we're gonna build up the dataset for training of the LLM consisting of many different smaller datasets where we sample subsets

In [45]:
n = 10
rarit_dataset_buffer = []
k=3
options_enabled = True

## Open domain questioning

In [46]:
datasets_openqa = ["tau/commonsense_qa", "math_qa", "web_questions", "wiki_qa", "yahoo_answers_qa", "freebase_qa", "ms_marco"]

In [25]:
dataset = load_dataset(datasets_openqa[4], split="train", streaming=True)
shuffled = iter(dataset.shuffle(buffer_size=10_000, seed=2024))
next(shuffled)


{'id': '2336421',
 'question': 'how can i mak my website visitors to come back soon and may not leave the website ?i want 10 charactris?',
 'answer': "It appears that the question period has expired. If you have received an answer that meets your needs, please choose a 'best answer.'. Regards.",
 'nbestanswers': ["It appears that the question period has expired. If you have received an answer that meets your needs, please choose a 'best answer.'. Regards.",
  "1. Good information. . 97% use the Internet to become better informed and, according to Forrester Research, high quality content is the most popular reason for returning to a site. Including informative pages on a website adds value, making it more than simply an online brochure or shop window. If you are retailing products, include information on after care or answer frequently asked questions (FAQs) on a dedicated page. A site that provides useful information sticks in a user's mind and will be. revisited in the future.. . 2. U

### Ready datasets

#### tau/commonsense_qa

In [32]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_openqa[0]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    options = ""
    if options_enabled:
        options = "\n Options: "
    answer = -1
    for idx, o in enumerate(example["choices"]["label"]):
        if options_enabled:
            options += f"{o}: {example['choices']['text'][idx]}, "
        if example["answerKey"] == o:
            answer = idx
    if options_enabled: 
        options += "\n"

    query = example["question"] + options
    if options_enabled:
        prediction = f"{example['answerKey']}) {example['choices']['text'][answer]}"
    else:
        prediction =  f"{example['choices']['text'][answer]}"
    example_id = example["id"] + "_" + example["question_concept"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="mc", domain="openqa")
    
    save_examples(i, start, last_time, examples, dname)

rarit_dataset_buffer

Processed 0 tau/commonsense_qa examples, time: 0:00:00, last 100 in 0:00:00.041352


[{'split': 'llm',
  'query': "The ficus tree isn't native to the state of actors and surfers but you can find them where?\n Options: A: front yard, B: california, C: farm, D: conservatory, E: tropical forest, \n",
  'prediction': 'B) california',
  'context': 'Ficus ficus is found in the Indian Ocean and the West Pacific.',
  'src': 'tau/commonsense_qa',
  'id': '41dfc69ea81b7e7ab1cceb690a12b6b0_ficus',
  'context_src': 'wiki',
  'context_id': 'Ficus ficus',
  'original_context': False,
  'task': 'mc',
  'domain': 'openqa'},
 {'split': 'llm',
  'query': "The ficus tree isn't native to the state of actors and surfers but you can find them where?\n Options: A: front yard, B: california, C: farm, D: conservatory, E: tropical forest, \n",
  'prediction': 'B) california',
  'context': 'Fact #1127: The tallest tree, the largest tree (by volume) and the oldest tree (non-clonal) in the world are all located in California.',
  'src': 'tau/commonsense_qa',
  'id': '41dfc69ea81b7e7ab1cceb690a12b6

#### math_qa

In [28]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_openqa[1]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)
    options = ""
    if options_enabled:
        options =  "\nOptions: " + example["options"]
    query = example["Problem"] + options

    answer = ""
    if options_enabled:
        answer = example["correct"] + ") "

    # Extract actual answer from options
    for idx, o in enumerate(example["options"].split(",")):
        a = o.split(" ) ")
        if example["correct"] == a[0].strip():
            answer += a[1]

    prediction = answer
    example_id = example["category"] + "_" + str(i)
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="mc", domain="openqa")
    
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 math_qa examples, time: 0:00:00, last 100 in 0:00:00.056603


[{'split': 'llm',
  'query': 'a grocer is storing soap boxes in cartons that measure 25 inches by 42 inches by 60 inches . if the measurement of each soap box is 7 inches by 12 inches by 5 inches , then what is the maximum number of soap boxes that can be placed in each carton ?\nOptions: a ) 210 , b ) 150 , c ) 280 , d ) 300 , e ) 420',
  'prediction': 'b) 150 ',
  'context': 'I was wondering if you could make a box that holds soap that measures 4×3.5 I have no idea of how to construct one of these, please please please help me…LOL!\nI just love the above box but I wondered if it will hold 6, 2×2″ brownies? If not then can you plz give me a measurement for a box of that size. Please need ur help on this',
  'src': 'math_qa',
  'id': 'general_1',
  'context_src': 'oscar',
  'context_id': 5814499,
  'original_context': False,
  'task': 'mc',
  'domain': 'openqa'},
 {'split': 'llm',
  'query': 'a grocer is storing soap boxes in cartons that measure 25 inches by 42 inches by 60 inches . i

In [40]:
rarit

,split,query,prediction,context,src,id,context_src,context_id,original_context,task,domain
0,llm,what team does allen iverson play for now?,Detroit Pistons,Guard Allen Iverson info and blog posts. Ivers...,web_questions,http://www.freebase.com/view/en/allen_iverson_...,oscar,3428772,False,qa,openqa
1,llm,what team does allen iverson play for now?,Detroit Pistons,"James D. Iverson (August 22, 1930 – October 26...",web_questions,http://www.freebase.com/view/en/allen_iverson_...,wiki,Jim Iverson,False,qa,openqa
2,llm,what team does allen iverson play for now?,Detroit Pistons,Mar 26 - Welcome to the first version of The A...,web_questions,http://www.freebase.com/view/en/allen_iverson_...,oscar,7318229,False,qa,openqa


#### web_questions

In [47]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_openqa[2]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    prediction =  example["answers"][0]
    example_id = example["url"] + "_" + example["answers"][0]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="qa", domain="openqa")
    
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=100.0)"), '(Request ID: d308aad2-b2a6-4288-b14e-f844f1382a45)')

#### Wiki QA

In [20]:
{'question_id': 'Q328',
 'question': 'WHAT COUNTRY IS MEXICO IN',
 'document_title': 'Mexico',
 'answer': 'It has the fourteenth largest nominal GDP and the eleventh largest GDP by purchasing power parity .',
 'label': 0}

{'question_id': 'Q328',
 'question': 'WHAT COUNTRY IS MEXICO IN',
 'document_title': 'Mexico',
 'answer': 'It has the fourteenth largest nominal GDP and the eleventh largest GDP by purchasing power parity .',
 'label': 0}

In [24]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_openqa[3]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    prediction =  example["answer"]
    example_id = example["question_id"] + "_" + example["document_title"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="qa", domain="openqa")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 wiki_qa examples, time: 0:00:00, last 100 in 0:00:03.718402


[{'split': 'llm',
  'query': 'what team does allen iverson play for now?',
  'prediction': 'Detroit Pistons',
  'context': "Guard Allen Iverson info and blog posts. Iverson is a professional basketball player in the NBA who currently plays for the Detroit Pistons. Follow Iverson here.\nRelationship dating details of Allen Iverson and Kenya Moore and all the other celebrities they've hooked up with.\nNBA baller Allen Iverson has got a new girlfriend. I just learned that the basketball all star has started dating former America's Next Top Model …\nAnd all these years later, the rant remains ubiquitous in society—from rap songs to shoe commercials to a judge's federal",
  'src': 'web_questions',
  'id': 'http://www.freebase.com/view/en/allen_iverson_Detroit Pistons',
  'context_src': 'oscar',
  'context_id': 3428772,
  'original_context': False,
  'task': 'qa',
  'domain': 'openqa'},
 {'split': 'llm',
  'query': 'what team does allen iverson play for now?',
  'prediction': 'Detroit Piston

#### yahoo answers qa

In [28]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_openqa[4]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    prediction =  example["answer"]
    example_id = example["id"] + "_" + example["main_category"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="qa", domain="openqa")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 yahoo_answers_qa examples, time: 0:00:00, last 100 in 0:00:26.776578


[{'split': 'llm',
  'query': 'what team does allen iverson play for now?',
  'prediction': 'Detroit Pistons',
  'context': "Guard Allen Iverson info and blog posts. Iverson is a professional basketball player in the NBA who currently plays for the Detroit Pistons. Follow Iverson here.\nRelationship dating details of Allen Iverson and Kenya Moore and all the other celebrities they've hooked up with.\nNBA baller Allen Iverson has got a new girlfriend. I just learned that the basketball all star has started dating former America's Next Top Model …\nAnd all these years later, the rant remains ubiquitous in society—from rap songs to shoe commercials to a judge's federal",
  'src': 'web_questions',
  'id': 'http://www.freebase.com/view/en/allen_iverson_Detroit Pistons',
  'context_src': 'oscar',
  'context_id': 3428772,
  'original_context': False,
  'task': 'qa',
  'domain': 'openqa'},
 {'split': 'llm',
  'query': 'what team does allen iverson play for now?',
  'prediction': 'Detroit Piston

## Reading comprehension

In [12]:
datasets_reading = ["coqa", "drop", "narrativeqa", "newsqa", ("pubmed_qa", "pqa_unlabeled"), "quail", "quarel", "squad_v2"] # "natural_questions", "trivia_qa",  "search_qa", "", "duorc", "ropes"

In [116]:
dreading = load_dataset(datasets_reading[4][0], datasets_reading[4][1], split="train")
rshuffled = iter(dreading.shuffle(seed=2024))
sample = next(rshuffled)

# len(sample["document"]["summary"]["tokens"])
len((" ".join(sample["context"]["contexts"])).split()) 

232

In [108]:
example["question"]

'When was the age limit changed?'

In [84]:
len(next(rshuffled)["document"]["summary"]["tokens"])

958

#### CoQA

In [99]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_reading[0]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["questions"][0]
    prediction =  example["answers"]['input_text'][0]
    example_id = example["source"] + "_" + str(i)
    context = example["story"]
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 coqa examples, time: 0:00:00, last 100 in 0:00:00.047865


[{'split': 'llm',
  'query': "What is Rooney's profession?",
  'prediction': "Sailor, missionary, Yes, a clasp-knife, No, tied them up, one of the fellows with him, No, on the mainland, before, to Godhaab, Yes, Yes, He's dying, a bear, a girl, Yes, It's his daughter, Yes, from the far north, near him, No, No, It's not safe., Food, Yes, at once",
  'context': 'Rooney',
  'src': 'drop',
  'id': 'gutenberg_0',
  'context_src': 'wiki',
  'context_id': 'Addicted Tour',
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': "What is Rooney's profession?",
  'prediction': "Sailor, missionary, Yes, a clasp-knife, No, tied them up, one of the fellows with him, No, on the mainland, before, to Godhaab, Yes, Yes, He's dying, a bear, a girl, Yes, It's his daughter, Yes, from the far north, near him, No, No, It's not safe., Food, Yes, at once",
  'context': 'Rooney',
  'src': 'drop',
  'id': 'gutenberg_0',
  'context_src': 'oscar',
  'context_id': 3432035,
  'or

#### drop

In [102]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
# This dataset seems to have a lot of duplicates so we remove them
dname = datasets_reading[1]
dataset = load_dataset(dname, split="train").to_pandas()
dataset.drop_duplicates(subset=["section_id"], keep="first", inplace=True)
dataset = Dataset.from_pandas(dataset)
shuffled = iter(dreading.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    spans = example["answers_spans"]['spans']
    if len(spans) > 1:
        prediction =  ", ".join(spans)
    else:
        prediction = spans[0]
    
    example_id = example["query_id"]
    context = example["passage"]
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 drop examples, time: 0:00:00, last 100 in 0:00:00.065023


[{'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9903101,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9478669,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': 'At the beginning of 1826.',
  'src': 'drop',
  'id': '12cbbb8e-673

#### narrativeqa

In [109]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
# This dataset seems to have a lot of duplicates so we remove them
dname = datasets_reading[2]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]["text"]
    prediction =  example["answers"][0]['text']
    example_id = example["document"]["id"] + "_" + str(i)
    context = example["document"]["summary"]["text"]
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 narrativeqa examples, time: 0:00:00, last 100 in 0:00:04.128370


[{'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9903101,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9478669,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': 'At the beginning of 1826.',
  'src': 'drop',
  'id': '12cbbb8e-673

#### newsqa - not working

In [ ]:
# Unfinished

dname = datasets_reading[3]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]["text"]
    prediction =  example["answers"][0]['text']
    example_id = example["document"]["id"] + "_" + str(i)
    context = example["document"]["summary"]["text"]
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

#### Pubmed qa

In [117]:
dname = datasets_reading[4][0]
dataset = load_dataset(dname,  datasets_reading[4][1], split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    prediction =  example["long_answer"]
    example_id = example["pubid"]
    context = " ".join(sample["context"]["contexts"])
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 pubmed_qa examples, time: 0:00:00, last 100 in 0:00:02.981752


[{'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9903101,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9478669,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': 'At the beginning of 1826.',
  'src': 'drop',
  'id': '12cbbb8e-673

#### quail

In [120]:
dname = datasets_reading[5]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    prediction =  example["answers"][example["correct_answer_id"]]
    if example["question_type"] == "Unanswerable":
        prediction = "I don't know."
    example_id = example["id"]
    context = " ".join(sample["context"])
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Generating challenge split: 100%|██████████| 556/556 [00:00<00:00, 148376.47 examples/s]


Processed 0 quail examples, time: 0:00:00, last 100 in 0:00:02.883202


[{'split': 'llm',
  'query': 'When did Moore allegedly preform his sexual misconduct?',
  'prediction': "In his 30's.",
  'context': "Moore's downfall began with a Washington Post article on Nov. 9 that alleged he pursued a sexual encounter with a 14-year-old girl when he was 32. In the Post’s article, a total of four women accused Moore of sexual misconduct in the late 1970s and early 1980s, and several more women came forward since.\nAlthough Moore repeatedly denied the claims, 51 percent of voters believed the accusations were true, according to the analysis.\nBut Moore also didn’t help himself when pressed on the issue during an interview with Fox News’ Sean Hannity.",
  'src': 'quail',
  'id': 'n073_4',
  'context_src': 'oscar',
  'context_id': 879025,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'When did Moore allegedly preform his sexual misconduct?',
  'prediction': "In his 30's.",
  'context': 'The Washington Post reported last 

#### quarel

In [122]:
dname = datasets_reading[6]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"].split("(A)")[0]
    if options_enabled:
        query += "\n Options: (A)" + example["question"].split("(A)")[1]

    answers = example["question"].split("(A)")[1].split("(B)")
    prediction =  answers[example["answer_index"]].strip()
    example_id = example["id"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="mc", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Generating validation split: 100%|██████████| 278/278 [00:00<00:00, 16666.90 examples/s]


Processed 0 quarel examples, time: 0:00:00, last 100 in 0:00:02.836173


[{'split': 'llm',
  'query': 'Evelyn lets go of a balloon and watches as it gets higher in the sky. When will it have looked larger \n Options: (A) closer to Evelyn (B) further from Evelyn',
  'prediction': ' closer to Evelyn ',
  'context': 'I have seen so many new things in the world because of Evelyn. Especially supermarket ceilings. Not that it’s particularly exciting, but I had never noticed them before. Evelyn does though, she is always looking up, or down, or turning her head sideways to see what the world looks like. Her new thing is to stand up, put her head on the ground and view the world between her legs.',
  'src': 'quarel',
  'id': 'QuaRel_V1_B5_0450',
  'context_src': 'oscar',
  'context_id': 7306340,
  'original_context': False,
  'task': 'mc',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'Evelyn lets go of a balloon and watches as it gets higher in the sky. When will it have looked larger \n Options: (A) closer to Evelyn (B) further from Evelyn',
  'prediction': ' c

#### squad v2

In [133]:
# Counting the unswerable questions in the dataset
dname = datasets_reading[7]
print(dname)
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))

unanswerable = 0
for i in range(15000):
    example = next(shuffled)
    if len(example["answers"]["text"]) == 0:
        unanswerable += 1
print(unanswerable)
        

squad_v2
4955


In [132]:
dname = datasets_reading[7]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n*2):
    example = next(shuffled)

    query = example["question"]
    answer = example["answers"]["text"]
    prediction = ""
    if len(answer) == 0:
        prediction = "I don't know."
    else:
        prediction = answer[0]
    example_id = example["id"]
    context = " ".join(sample["context"])
    examples = make_examples(query, prediction, dname, example_id=example_id, context=context, k=k, retrieval=True, task="qa", domain="rc")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 squad_v2 examples, time: 0:00:00, last 100 in 0:00:01.791357


[{'split': 'llm',
  'query': 'Until what year did the Ottomans retain a band of territory including Albania, Macedonia and Thrace?',
  'prediction': '1912',
  'context': "Two Balkan Wars, in 1912 and 1913, entailed further action against the Ottoman Empire in Europe. The Balkan League first conquered Macedonia and most of Thrace from the Ottoman Empire, and then fell out over the division of the spoils. Albania declared its independence from the Ottoman Empire in 1912, after several rebellions and uprisings. This reduced Turkey's possessions in Europe (Rumelia) to their present borders in Eastern Thrace.",
  'src': 'squad_v2',
  'id': '56f8da749b226e1400dd10f3',
  'context_src': 'wiki',
  'context_id': 'Ottoman wars in Europe',
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'Until what year did the Ottomans retain a band of territory including Albania, Macedonia and Thrace?',
  'prediction': '1912',
  'context': 'By the 1890s only Macedonia

## Summarization

In [30]:
datasets_summ = ["cnn_dailymail"]

In [33]:
dataset = load_dataset(datasets_summ[0], '1.0.0', split="train", streaming=True)
shuffled = iter(dataset.shuffle(buffer_size=10_000, seed=2024))
s = next(shuffled)
s.keys()

dict_keys(['article', 'highlights', 'id'])

In [103]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = "cnn_dailymail"
dataset = load_dataset(dname, "1.0.0", split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = "Summarize this article"
    prediction =  example["highlights"]
    example_id = example["id"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, context=example["article"], retrieval=False, task="sum", domain="sum")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 72504.24 examples/s]


Processed 0 cnn_dailymail examples, time: 0:00:00, last 100 in 0:00:00.000343


[{'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9903101,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': '1826.',
  'src': 'drop',
  'id': '12cbbb8e-673a-436c-ab3a-553ad180df38',
  'context_src': 'oscar',
  'context_id': 9478669,
  'original_context': False,
  'task': 'qa',
  'domain': 'rc'},
 {'split': 'llm',
  'query': 'What three men started businesses or established a business headquarters in 1826?',
  'prediction': 'Jacob Gabel, Robert Douglas, John Galt',
  'context': 'At the beginning of 1826.',
  'src': 'drop',
  'id': '12cbbb8e-673

## Chain of thought

In [4]:
datasets_cot = ["aqua_rat", "yangdong/ecqa", "gsm8k", "hendrycks/competition_math", "metaeval/strategy-qa"]

In [21]:
dataset = load_dataset(datasets_cot[3], split="train")
shuffled = iter(dataset.shuffle(seed=2024))
s = next(shuffled)
s

{'problem': 'Consider the following function $g(x)$ defined as\\[(x^{2^{2008}-1}-1)g(x) = (x+1)(x^2+1)(x^4+1)\\cdots (x^{2^{2007}}+1) - 1\\]Find $g(2)$.\n',
 'level': 'Level 5',
 'type': 'Intermediate Algebra',
 'solution': 'Multiply both sides by $x-1$; the right hand side collapses by the reverse of the difference of squares.\n\\begin{align*}(x-1)(x^{2^{2008}-1}-1)g(x) &= (x-1)(x+1)(x^2+1)(x^4+1)\\cdots (x^{2^{2007}}+1) - (x-1)\\\\ &= (x^2-1) (x^2+1)(x^4+1)\\cdots (x^{2^{2007}}+1) - (x-1)\\\\ &= \\cdots\\\\ &= \\left(x^{2^{2008}}-1\\right) - (x-1) = x^{2^{2008}} - x \\end{align*}Substituting $x = 2$, we have\\[\\left(2^{2^{2008}-1}-1\\right) \\cdot g(2) = 2^{2^{2008}}-2 = 2\\left(2^{2^{2008}-1}-1\\right)\\]Dividing both sides by $2^{2^{2008}-1}$, we find $g(2) = \\boxed{2}$.'}

In [23]:
next(shuffled)

{'problem': 'Koalas absorb only $25\\%$ of the fiber they eat. A koala absorbed 10.5 ounces of fiber in one day. How many ounces of fiber did he eat that day?',
 'level': 'Level 3',
 'type': 'Prealgebra',
 'solution': 'We seek the answer to the question "10.5 ounces is 25% of what number?" If we call the unknown number of ounces $x$, we have the equation $10.5=0.25x$. Dividing both sides by $0.25$, we have $x=\\frac{10.5}{0.25}=\\boxed{42}$ ounces of fiber.'}

#### aqua_rat

In [49]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_cot[0]
dataset = load_dataset(dname, "raw", split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"] + "\n" + example["rationale"]

    options = "\nOptions: "
    correct = -1
    for idx, o in enumerate(example["options"]):
        options += f"\n{o}"
        if o[0] == example["correct"]:
            correct = idx

    if options_enabled:
        query += options
        prediction = example["options"][correct]   
    else:
        prediction = example["options"][correct].split(")")[1]

    example_id = dname + "_" + str(i)
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="mc", domain="cotr")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

#### ecqa

In [15]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_cot[1]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["q_text"]
    query += "\n" + example["taskA_pos"]

    options = "\nOptions: "
    correct = -1
    for idx in range(5):
        options += f"{idx+1}) {example[f'q_op{idx+1}']}"
        if example["q_ans"] == example[f'q_op{idx+1}']:
            correct = idx + 1
    if options_enabled:
        query += options
        prediction = f"{correct}) {example[f'q_op{correct}']}"
    else:
        prediction = example[f'q_op{correct}']
    
    example_id = example["q_no"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="mc", domain="cotr")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 yangdong/ecqa examples, time: 0:00:00, last 100 in 0:00:02.643672


[{'split': 'llm',
  'query': 'A boy multiplied 987 by a certain number and obtained 559981 as his answer. If in the answer both 98 are wrong and other digits are correct, then the correct answer would be:\n987 = 3 x 7 x 47.\nSo, the required number must be divisible by each one of 3, 7, 47\n553681 --> sum of digits = 28, not divisible by 3\n555181 sum of digits = 25, not divisible by 3\n555681 is divisible by each one of 3, 7, 47.\n556581 sum of digits = 30, not divisible by 7\n555581 sum of digits = 29, not divisible by 3\nso the answer is C.',
  'prediction': 'C)555681',
  'context': '6 × 7 × 8 = 336 ; 3) 336 ÷ 3 = 112 First, take any number (for this example it will be 492) and add together each digit in the number (4 + 9 + 2 = 15). Then take that sum (15) and determine if it is divisible by 3. The original number is divisible by 3 (or 9) if and only if the sum of its digits is divisible by 3 (or 9). Adding the digits of a number up, and then repeating the process with the result un

#### gsm8k

In [17]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_cot[2]
dataset = load_dataset(dname, "main", split="train", streaming=True)
shuffled = iter(dataset.shuffle(buffer_size=10_000, seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    rational = example["answer"].split("####")
    query = example["question"] + "\n" + rational[0]
    prediction =  rational[1]
    example_id = dname + "_" + str(i)
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="qa", domain="cotr")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 gsm8k examples, time: 0:00:00, last 100 in 0:00:03.079255


[{'split': 'llm',
  'query': 'A boy multiplied 987 by a certain number and obtained 559981 as his answer. If in the answer both 98 are wrong and other digits are correct, then the correct answer would be:\n987 = 3 x 7 x 47.\nSo, the required number must be divisible by each one of 3, 7, 47\n553681 --> sum of digits = 28, not divisible by 3\n555181 sum of digits = 25, not divisible by 3\n555681 is divisible by each one of 3, 7, 47.\n556581 sum of digits = 30, not divisible by 7\n555581 sum of digits = 29, not divisible by 3\nso the answer is C.',
  'prediction': 'C)555681',
  'context': '6 × 7 × 8 = 336 ; 3) 336 ÷ 3 = 112 First, take any number (for this example it will be 492) and add together each digit in the number (4 + 9 + 2 = 15). Then take that sum (15) and determine if it is divisible by 3. The original number is divisible by 3 (or 9) if and only if the sum of its digits is divisible by 3 (or 9). Adding the digits of a number up, and then repeating the process with the result un

#### math

In [26]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_cot[3]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["problem"]
    prediction =  example["solution"]
    example_id = dname + "_" + str(i)
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="qa", domain="openqa")
        
    save_examples(i, start, last_time, examples, dname)
rarit_dataset_buffer

Processed 0 hendrycks/competition_math examples, time: 0:00:00, last 100 in 0:00:01.314626


[{'split': 'llm',
  'query': 'A boy multiplied 987 by a certain number and obtained 559981 as his answer. If in the answer both 98 are wrong and other digits are correct, then the correct answer would be:\n987 = 3 x 7 x 47.\nSo, the required number must be divisible by each one of 3, 7, 47\n553681 --> sum of digits = 28, not divisible by 3\n555181 sum of digits = 25, not divisible by 3\n555681 is divisible by each one of 3, 7, 47.\n556581 sum of digits = 30, not divisible by 7\n555581 sum of digits = 29, not divisible by 3\nso the answer is C.',
  'prediction': 'C)555681',
  'context': '6 × 7 × 8 = 336 ; 3) 336 ÷ 3 = 112 First, take any number (for this example it will be 492) and add together each digit in the number (4 + 9 + 2 = 15). Then take that sum (15) and determine if it is divisible by 3. The original number is divisible by 3 (or 9) if and only if the sum of its digits is divisible by 3 (or 9). Adding the digits of a number up, and then repeating the process with the result un

#### strategyqa

In [30]:
# def make_examples(query: str, prediction:str, dataset_name:str, context = None, example_id = None, k: int = 3, retrieval = True)
dname = datasets_cot[4]
dataset = load_dataset(dname, split="train")
shuffled = iter(dataset.shuffle(seed=2024))
start = time.time()
last_time = start
for i in range(n):
    example = next(shuffled)

    query = example["question"]
    rational = ""
    for idx, de in enumerate(example["decomposition"]):
        if len(example["facts"]) > idx:
            rational += f"{de} {example['facts']} "
        else:
            rational += f"{de}"
    query += "\n" + rational
    prediction =  example["answer"]
    example_id = example["qid"]
    examples = make_examples(query, prediction, dname, example_id=example_id, k=k, retrieval=True, task="qa", domain="openqa")
        
    save_examples(i, start, last_time, examples, dname)
# Force save after the last example
save_examples(0, start, last_time, [], dname, force=True)
rarit_dataset_buffer

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1330.26it/s]
Generating train split: 2290 examples [00:00, 90612.79 examples/s]


Processed 0 metaeval/strategy-qa examples, time: 0:00:00, last 100 in 0:00:03.343660


[{'split': 'llm',
  'query': 'A boy multiplied 987 by a certain number and obtained 559981 as his answer. If in the answer both 98 are wrong and other digits are correct, then the correct answer would be:\n987 = 3 x 7 x 47.\nSo, the required number must be divisible by each one of 3, 7, 47\n553681 --> sum of digits = 28, not divisible by 3\n555181 sum of digits = 25, not divisible by 3\n555681 is divisible by each one of 3, 7, 47.\n556581 sum of digits = 30, not divisible by 7\n555581 sum of digits = 29, not divisible by 3\nso the answer is C.',
  'prediction': 'C)555681',
  'context': '6 × 7 × 8 = 336 ; 3) 336 ÷ 3 = 112 First, take any number (for this example it will be 492) and add together each digit in the number (4 + 9 + 2 = 15). Then take that sum (15) and determine if it is divisible by 3. The original number is divisible by 3 (or 9) if and only if the sum of its digits is divisible by 3 (or 9). Adding the digits of a number up, and then repeating the process with the result un